In [619]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
import numpy as np
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import pickle
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
import operator

In [620]:
insulin_data = pd.DataFrame(pd.read_csv('insulinData.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (13,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [621]:
cgm_data = pd.DataFrame(pd.read_csv('CGMData.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [622]:
match_time = []
match_date = []
intake = []
bolus = []
for i in range(len(insulin_data)):
    if((not(math.isnan(insulin_data.loc[i, 'BWZ Carb Input (grams)']))) and (insulin_data.loc[i, 'BWZ Carb Input (grams)'] != 0)):
        match_date.append(insulin_data.loc[i, 'Date'])
        match_time.append(insulin_data.loc[i, 'Time'])
        intake.append(insulin_data.loc[i, 'BWZ Carb Input (grams)'])
        bolus.append(round(insulin_data.loc[i, 'BWZ Estimate (U)']))

In [623]:
dtl = []
for i in range(len(match_date)):
    dtl.append(str(match_date[i]) + ' ' + str(match_time[i]))

In [624]:
dtl_in = []
for i in range(len(intake)):
    dtl_in.append(intake[i])

In [625]:
dtl_bolus = []
for i in range(len(bolus)):
    dtl_bolus.append(bolus[i])

In [626]:
insulin_data = insulin_data.iloc[::-1]

In [627]:
cgm_data = cgm_data[::-1]

In [628]:
for i in range(len(dtl)):
    dtl[i] = datetime.strptime(str(match_date[i]) + ' ' + str(match_time[i]), '%m/%d/%Y %H:%M:%S')


In [629]:
dtl = dtl[::-1]

In [630]:
dtl_in = dtl_in[::-1]

In [631]:
dtl_bolus = dtl_bolus[::-1]

In [632]:
meal_data_dt = []
meal_data_dt_in = []
meal_data_dt_bolus = []
for i in range(len(dtl)-1):
    if((dtl[i] + timedelta(hours = 2)) < dtl[i+1]):
        meal_data_dt.append(dtl[i])
        meal_data_dt_in.append(dtl_in[i])
        meal_data_dt_bolus.append(dtl_bolus[i])
    if((dtl[i] + timedelta(hours = 2)) == dtl[i+1]):
        meal_data_dt.append(dtl[i+1] + timedelta(hours = -0.5))
        meal_data_dt_in.append(dtl_in[i+1])
        meal_data_dt_bolus.append(dtl_bolus[i+1])

In [633]:
cgm_data_dt = []
for i in range(len(cgm_data)):
    cgm_data_dt.append(datetime.strptime(str(cgm_data.loc[i, 'Date']) + ' ' + str(cgm_data.loc[i, 'Time']), '%m/%d/%Y %H:%M:%S'))

# meal data datetime of cgm


In [634]:
cgm_tm_match_meal = []
cgm_tm_match_meal_in = []
cgm_tm_match_meal_bolus = []
for i in range(len(meal_data_dt)):
    for j in range(len(cgm_data_dt)):
        if(cgm_data_dt[j] < meal_data_dt[i] + timedelta(minutes = 5)):
            cgm_tm_match_meal.append((cgm_data_dt[j]))
            cgm_tm_match_meal_in.append(meal_data_dt_in[i])
            cgm_tm_match_meal_bolus.append(meal_data_dt_bolus[i])
            break

In [635]:
cgm_data['timestamp'] = cgm_data_dt[::-1]

# Meal data cgm values

In [636]:
temp_index_list = []
temp_index_list_in = []
temp_index_list_bolus = []
for i in range(len(cgm_tm_match_meal)):
    temp_index_list.append(cgm_data[cgm_data['timestamp'] == cgm_tm_match_meal[i]].index[0])
    temp_index_list_in.append(cgm_tm_match_meal_in[i])
    temp_index_list_bolus.append(cgm_tm_match_meal_bolus[i])

In [637]:
cgm_value_stretch_meal = []
cgm_value_stretch_meal_in = []
cgm_value_stretch_meal_bolus = []
for i in range(1,len(temp_index_list)):
    cgm_value_one_stretch_meal = []
    for j in range(30):
        cgm_value_one_stretch_meal.append(cgm_data.loc[(temp_index_list[i] + 6 - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_meal).all())):
        cgm_value_stretch_meal.append(cgm_value_one_stretch_meal)
        cgm_value_stretch_meal_in.append(temp_index_list_in[i])
        cgm_value_stretch_meal_bolus.append(temp_index_list_bolus[i])

In [638]:
for i in range(len(cgm_value_stretch_meal)):
    for j in range(len(cgm_value_stretch_meal[i])):
        if(np.isnan(cgm_value_stretch_meal[i][j])):
            cgm_value_stretch_meal[i][j] = int(np.nanmean(cgm_value_stretch_meal[i]))

In [639]:
cgm_value_stretch_meal_df = pd.DataFrame(cgm_value_stretch_meal)

In [640]:
cgm_value_stretch_meal_df['intake'] = cgm_value_stretch_meal_in

In [641]:
cgm_value_stretch_meal_df['bolus'] = (cgm_value_stretch_meal_bolus)

In [642]:
cgm_max = max(cgm_value_stretch_meal_df.max())

In [643]:
cgm_min = min(cgm_value_stretch_meal_df.min())

In [644]:
bins = (cgm_max - cgm_min )/ 20

In [645]:
b_meal_class_list = []
for i in range(len(cgm_value_stretch_meal_df)):
    b_meal_class_list.append(np.ceil((cgm_value_stretch_meal_df[6])[i]/20))
        
            

In [646]:
b_max_class_list = []
for i in range(len(cgm_value_stretch_meal_df)):
    b_max_class_list.append(np.ceil((max(cgm_value_stretch_meal_df.loc[i][:-2]/20))))
        

In [647]:
cgm_value_stretch_meal_df['b_max'] = b_max_class_list

In [648]:
cgm_value_stretch_meal_df['b_meal'] = b_meal_class_list

In [649]:
items = pd.DataFrame(cgm_value_stretch_meal_df[['bolus', 'b_max', 'b_meal']])

# Bolus

In [650]:
bolus_freq = items['bolus'].value_counts().to_dict()

In [651]:
b_max_freq = items['b_max'].value_counts().to_dict()

In [652]:
b_meal_freq = items['b_meal'].value_counts().to_dict()

In [653]:
bolus_bmax_bmeal = (items.groupby(['bolus', 'b_max', 'b_meal']).size()).to_dict()

In [654]:
bolus_bmax = (items.groupby(['bolus', 'b_max']).size()).to_dict()

In [655]:
bolus_bmeal = (items.groupby(['bolus', 'b_meal']).size()).to_dict()

In [656]:
bmax_bmeal = (items.groupby(['b_max', 'b_meal']).size()).to_dict()

In [657]:
bolus = (items.groupby(['bolus']).size()).to_dict()

In [658]:
bmax = (items.groupby(['b_max']).size()).to_dict()

In [659]:
bmeal = (items.groupby(['b_meal']).size()).to_dict()

In [660]:
mfreq_bolus_bmax_bmeal = max(bolus_bmax_bmeal.items(), key=operator.itemgetter(1))[0]

In [661]:
mfreq_bolus_bmax = max(bolus_bmax.items(), key=operator.itemgetter(1))[0]

In [662]:
mfreq_bolus_bmeal = max(bolus_bmeal.items(), key=operator.itemgetter(1))[0]

In [663]:
mfreq_bmax_bmeal = max(bmax_bmeal.items(), key=operator.itemgetter(1))[0]

In [664]:
mfreq_bolus = max(bolus.items(), key=operator.itemgetter(1))[0]

In [665]:
mfreq_bmax = max(bmax.items(), key=operator.itemgetter(1))[0]

In [666]:
mfreq_bmeal = max(bmeal.items(), key=operator.itemgetter(1))[0]

In [667]:
line1 = str(mfreq_bolus)

In [668]:
line2 = str(mfreq_bmax)

In [669]:
line3 = str(mfreq_bmeal)

In [670]:
csv1_list = []
csv1_list.append(mfreq_bolus_bmax_bmeal)
csv1_list.append(mfreq_bolus_bmax)
csv1_list.append(mfreq_bolus_bmeal)
csv1_list.append(mfreq_bmax_bmeal)
csv1_list.append(line1)
csv1_list.append(line2)
csv1_list.append(line3)

In [671]:
csv1_list[4] = str(csv1_list[4])

In [672]:
csv1_list[5] = str(csv1_list[5])

In [673]:
csv1_list[6] = str(csv1_list[6])

In [674]:
dict1 = {'list1': csv1_list}

In [675]:
df1 = pd.DataFrame(dict1)

In [676]:
df1.to_csv(r'MURALI_results1_frequent_itemsets.csv', index = False, columns=None)

In [677]:
final_df1 = pd.read_csv("MURALI_results1_frequent_itemsets.csv", encoding="utf-8", skiprows=1)
final_df1.to_csv(r'.\MURALI_results1_frequent_itemsets.csv', index=False)

# confidence

In [678]:
final_conf_list = []
temp_list = []
for vali in bmax_bmeal:
    #temp_list = []
    for valj in bolus_bmax_bmeal:
        if((vali[0] == valj[1]) and (vali[1] == valj[2])):
            temp_list.append((valj[0], valj[1], valj[2], bolus_bmax_bmeal[valj[0], valj[1], valj[2]]))
    max_temp = max([temp_list_item[3]/bmax_bmeal[vali[0], vali[1]] for temp_list_item in temp_list])
    for i in (temp_list):
        if((i[3]/bmax_bmeal[vali[0], vali[1]] == max_temp)):
            final_conf_list.append(i)

In [679]:
full_conf_list = []
temp_list = []
for vali in bmax_bmeal:
    #temp_list = []
    for valj in bolus_bmax_bmeal:
        if((vali[0] == valj[1]) and (vali[1] == valj[2])):
            temp_list.append((valj[0], valj[1], valj[2], bolus_bmax_bmeal[valj[0], valj[1], valj[2]]))
    #max_temp = max([temp_list_item[3]/bmax_bmeal[vali[0], vali[1]] for temp_list_item in temp_list])
    for i in (temp_list):
        #if((i[3]/bmax_bmeal[vali[0], vali[1]] == max_temp)):
        full_conf_list.append((i, (i[3]/bmax_bmeal[vali[0], vali[1]])))

In [680]:
temp1 = []
for i in range(len(full_conf_list)):
    if(full_conf_list[i][1] < 0.15):
        (temp1.append((full_conf_list[i][0])))
full_conf_list_set = list(set(temp1))
        

In [681]:
large_conf_list = list(set(final_conf_list))

In [682]:
list2 = []
for i in large_conf_list:
    list2.append("{" + str(i[1]) + "," + str(i[2]) + "} -> " + str(i[0]))

In [683]:
df2 = pd.DataFrame(list2)

In [684]:
df2.to_csv(r'MURALI_results2_large_confidence.csv', index = False, columns=None)

In [685]:
final_df = pd.read_csv("MURALI_results2_large_confidence.csv", encoding="utf-8", skiprows=1)
final_df.to_csv(r'.\MURALI_results2_large_confidence.csv', index=False)

In [686]:
list3 = []
for i in full_conf_list_set:
    list3.append("{" + str(i[1]) + "," + str(i[2]) + "} -> " + str(i[0]))

In [687]:
df3 = pd.DataFrame(list3)

In [688]:
df3.to_csv(r'MURALI_results3_low_confidence.csv', index = False, columns=None)

In [689]:
final_df3 = pd.read_csv("MURALI_results3_low_confidence.csv", encoding="utf-8", skiprows=1)
final_df3.to_csv(r'.\MURALI_results3_low_confidence.csv', index=False)